In [1]:
import pandas as pd
from sqlalchemy import create_engine


In [2]:
csv_file = "twtr_clean_all.csv"
twtr_all = pd.read_csv(csv_file)
twtr_all.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2013-11-07,45.099998,50.090000,44.000000,44.900002,117701670,0.0,0.0
1,2013-11-08,45.930000,46.939999,40.685001,41.650002,27925307,0.0,0.0
2,2013-11-11,40.500000,43.000000,39.400002,42.900002,16113941,0.0,0.0
3,2013-11-12,43.660000,43.779999,41.830002,41.900002,6316755,0.0,0.0
4,2013-11-13,41.029999,42.869999,40.759998,42.599998,8688325,0.0,0.0


In [3]:
twtr_all_df = twtr_all[['Date', 'Open', 'High','Low','Close','Volume']].copy()
twtr_all_df.head()

,Date,Open,High,Low,Close,Volume
0,2013-11-07,45.099998,50.090000,44.000000,44.900002,117701670
1,2013-11-08,45.930000,46.939999,40.685001,41.650002,27925307
2,2013-11-11,40.500000,43.000000,39.400002,42.900002,16113941
3,2013-11-12,43.660000,43.779999,41.830002,41.900002,6316755
4,2013-11-13,41.029999,42.869999,40.759998,42.599998,8688325


In [4]:
json_file = "twtr_dates.json"
twtr_dates_df = pd.read_json(json_file)
twtr_dates_df.head()

,Date,Headline
0,2022-01-31,January 31: Musk begins building up his Tw...
1,2022-03-14,March 14: Musk’s Twitter stake tops 5%
2,2022-03-24,March 24: Asking whether Twitter should ch...
5,2022-04-04,April 4: Surprise! Musk becomes Twitter’s ...
6,2022-04-05,April 5: Musk agrees to join the board


In [5]:
import psycopg2

In [6]:
protocol = 'postgresql'
username = 'postgres'
password = 'postgres'
host = 'localhost'
port = 5432
database_name = 'twitter_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [7]:
engine.table_names()

['twtr_all', 'twtr_dates']

In [8]:
twtr_all_df.to_sql(name='twtr_all', con=engine, if_exists='append', index=False)

In [9]:
twtr_dates_df.to_sql(name='twtr_dates', con=engine, if_exists='append', index=False)

In [10]:
pd.read_sql_query('select * from twtr_all', con=engine).head()

,Date,Open,High,Low,Close,Volume
0,2013-11-07,45.0,50.0,44.0,45.0,117701670
1,2013-11-08,46.0,47.0,41.0,42.0,27925307
2,2013-11-11,41.0,43.0,39.0,43.0,16113941
3,2013-11-12,44.0,44.0,42.0,42.0,6316755
4,2013-11-13,41.0,43.0,41.0,43.0,8688325


In [11]:
pd.read_sql_query('select * from twtr_dates', con=engine).head()

,Date,Headline
0,2022-01-31,January 31: Musk begins building up his Tw...
1,2022-03-14,March 14: Musk’s Twitter stake tops 5%
2,2022-03-24,March 24: Asking whether Twitter should ch...
3,2022-04-04,April 4: Surprise! Musk becomes Twitter’s ...
4,2022-04-05,April 5: Musk agrees to join the board


In [38]:
sql_join = r"""SELECT twtr_all.Date, twtr_all.Open, twtr_all.High, twtr_all.Low, twtr_all.Close, twtr_all.Volume, twtr_dates.Headline
FROM twtr_all
JOIN twtr_dates
ON twtr_all.Date = twtr_dates.Date"""

In [39]:
pd.read_sql_query(sql_join, con=engine).head()

ProgrammingError: (psycopg2.errors.UndefinedColumn) column twtr_all.date does not exist
LINE 4: ON twtr_all.Date = twtr_dates.Date
           ^
HINT:  Perhaps you meant to reference the column "twtr_all.Date".

[SQL: SELECT twtr_all.Date, twtr_all.Open, twtr_all.High, twtr_all.Low, twtr_all.Close, twtr_all.Volume, twtr_dates.Headline
FROM twtr_all
JOIN twtr_dates
ON twtr_all.Date = twtr_dates.Date]
(Background on this error at: http://sqlalche.me/e/f405)

In [ ]:
SELECT twtr_all.Date, twtr_all.Open, twtr_all.High, twtr_all.Low, twtr_all.Close, twtr_all.Volume,twtr_dates.Headline
FROM twtr_all
inner join twtr_dates.Date = twtr_all.Date
limit(10);